In [1]:
"""
食材名を解析するスクリプトです。
"""
import pandas as pd
import MeCab
from tqdm import tqdm as progress

# 食材名データの読み込み
data = pd.read_csv("../data/recommend_ingredients.csv",names=["id","name","quantity"])
# 変換表の読み込み
exchange = pd.read_csv("../data/exchange_before.csv",names=["id","name","plus","unit","g"])
# カナに統一した変換表の読み込み
exchange_kana = pd.read_csv("../data/exchanged_map.csv")

# 必要な情報のみ抽出
exchange_kana = exchange_kana[["id","name"]]
data = data[["id","name"]]

m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

"""
食材名から名詞部分を抜き出す関数
名詞が一つの場合はその読みを返す。
入力が文字列でない場合はNoneを返す
複数の名詞がある場合には区切り文字を;にして連結する
名詞ではあるが、,を区切り文字とした場合に辞書の8番目に読みがない場合は
先頭に?をつけて返している。
名詞がない場合は-を返す
"""
def pick_ingredients(words,m):
  tmp_ingredients = []
  if type(words) != str:
    return None
  else:
    parsed_text = m.parseToNode(words)
    while parsed_text:
      node = parsed_text.feature.split(',')
      # 名詞のみ抽出
      if node[0] == "名詞":
        try:
          tmp_ingredients.append(node[7])
        except IndexError: # 8番目の要素に読みがない場合
          tmp_ingredients.append( "?" + parsed_text.surface)
      else:
        # 名詞以外は無視
        pass
      parsed_text = parsed_text.next
  length = len(tmp_ingredients)
  if length == 1:
    return tmp_ingredients[0]
  elif length >= 2:
    return ';'.join(tmp_ingredients)
  else:
    return '-'

# 進捗を確認
progress.pandas()
# 関数を適応
data_name_separated = data
data_name_separated["name"] = data['name'].progress_apply(pick_ingredients,m=m)
print(data_name_separated.head(20))

 56%|█████▌    | 6242489/11235222 [47:30<12:12:54, 113.54it/s]

Error: Session cannot generate requests